# GUI testing code V3 - multithreading

In [ ]:
def main():
    import numpy as np
    import pandas as pd
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.dates as mdates
    from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
    import PySimpleGUI as sg
    import pickle
    import time
    import datetime
    import threading
    import queue
    import importlib 
    import yfinance as yfi
    import regex as re


    my_func = importlib.import_module("Stock_portfolio_functions2")
    my_func = importlib.reload(my_func)

    
    ###################################################################################################
    ###-----------------------------------Retrieving needed data-----------------------------------####
    ###################################################################################################
    sg.theme('black')

    # Getting printout of all stocks for menu default 
    stock_info_dict = pickle.load(open('stock_info_dictionary.p','rb'))
    mega_stock_name_string = ''

    for key in stock_info_dict.keys():
        mega_stock_name_string = mega_stock_name_string + str(key) + ' - ' + str(stock_info_dict[key]['shortName']) + ' \n'

    ### Getting other background lists that contain industry information for filter
    industry_list_sorted_good2 = pickle.load(open('industry_list.p', 'rb'))
    industry_stocks_dict = pickle.load(open('stock_industry_dictionary.p','rb'))
    states_abrv = pd.read_csv('states.csv')

    portfolio_dict = pickle.load(open('stock_portfolio_dict_TEST.p', 'rb'))
    portfolio_price_list = []
    portfolio_stock_sym_list = []
    for stock in portfolio_dict:
        stock_sym = stock.split('~')[0]
        portfolio_price_list.append(yfi.Ticker(stock_sym).history().iloc[-1]['Close']*portfolio_dict[stock]['num_stocks'])
        portfolio_stock_sym_list.append(stock_sym)

    with open('money_made.txt','r', encoding='utf-8') as file:
        money_made = file.read()

    notified_analysis_broken = False

    
    
    ###################################################################################################
    ###---------------------------------------Portfolio tab-----------------------------------------###
    ###################################################################################################
    portfolio_combo1_list = []
    for stock in portfolio_dict:
        portfolio_combo1_list.append(stock)
    portfolio_combo1_list = list(np.sort(portfolio_combo1_list))

    text_list = my_func.update_portfolio_text()

    col1_portfolio = [
        [sg.Canvas(key="-CANVAS_PORTFOLIO-")],
        [sg.Multiline(text_list, expand_x=True, expand_y=True, background_color='black',
                     disabled=True, text_color='white', k='-MULTILINE_PORTFOLIO-')]
    ]

    col2_portfolio = [
        [sg.T('Performance of portfolio stocks', size=24, text_color='white', expand_x=True,
                justification='c')],
        [sg.Combo(values = portfolio_combo1_list, default_value='Portfolio',
                 readonly=True, key='-COMBO_PORTFOLIO-')],
        [sg.T('')],
        [sg.Canvas(key="-CANVAS_PORTFOLIO2-", border_width=10)]
    ]


    tab_portfolio_layout = [
        [sg.Frame('',col1_portfolio, element_justification='c', 
                   background_color='black', expand_y=True, vertical_alignment='top'),
        sg.Frame('',col2_portfolio, element_justification='c',
                  background_color='black', expand_y=False, vertical_alignment='top')]
    ]

    
    ##################################################################################################
    ###----------------------------------Stock Performance Tab-------------------------------------###
    ##################################################################################################
    frame1_performance = [
            [sg.Multiline('List of stocks: \n' + mega_stock_name_string, size=(25,3), expand_x=False, 
                          expand_y=False, k='-MLINE_PERFORMANCE-', pad=(5,5), disabled=True, 
                          auto_refresh=True), 
         sg.Combo(values=(industry_list_sorted_good2), default_value='All stocks', 
                  readonly=True, k='-COMBO_PERFORMANCE-'),
         ### Need to get list of all industry types
         sg.T('       '), ### spacer
         sg.Input(key='-INPUT_PERFORMANCE-', size=(25,1)), sg.Button(button_text='Search', 
                                                                     k='-SEARCH_PERFORMANCE-')]
    ]

    col1_performance = [
        [sg.Canvas(key="-CANVAS_PERFORMANCE-")]
    ]

    col2_performance = [
        [sg.Button('Today', expand_x=True, k='-BUTTON_PERFORMANCE_TODAY-')], 
        [sg.Button('1 Week', expand_x=True, k='-BUTTON_PERFORMANCE_1WEEK-')], 
        [sg.Button('1 Month', expand_x=True, k='-BUTTON_PERFORMANCE_1MONTH-')], 
        [sg.Button('6 Months', expand_x=True, k='-BUTTON_PERFORMANCE_6MONTHS-')], 
        [sg.Button('1 Year', expand_x=True, k='-BUTTON_PERFORMANCE_1YEAR-')]
    ]

    col3_performance = [
        [sg.T("Top 3 increasing stocks", auto_size_text=True,font=('Helvitica',30))],
        [sg.T('Text 1: x%'), sg.T('Text 2: x%')],
        [sg.T('Text 3: x%')],
        [sg.T('',background_color='red')],
        ### Here need to activate the python file that gets the top 3 performers. Can have progress bar 
        ### over it while it loads
        ### The text will be green and will have a + in front of it along with the stock names
        [sg.T("Top 3 decreasing stocks", auto_size_text=True,font=('Helvitica',30))],
        [sg.T('Text 1: x%'), sg.T('Text 2: x%')],
        [sg.T('Text 3: x%')]
        ### Here need to activate the python file that gets the top 3 performers. Can have progress bar
        ### over it while it loads
        ### The text will be red and have a - in front along with the stock names
    ]

    tab_performance_layout = [
        [sg.Frame('',frame1_performance, expand_x=True)],
        [sg.Column(col1_performance, justification='l', element_justification='c'),
        sg.Column(col2_performance, justification='l', element_justification='c'),
        sg.Column(col3_performance, justification='l', element_justification='c',
                 background_color='red')] 
    ]

    
    ###################################################################################################
    ###-----------------------------------Stock Analysis Tab----------------------------------------###
    ###################################################################################################
    ### Stock Analysis tab
    frame1_analysis = [
            [sg.Multiline('List of stocks: \n' + mega_stock_name_string, size=(25,3), expand_x=False, expand_y=False, 
                          k='-MLINE_ANALYSIS-', pad=(5,5), disabled=True), 
         sg.Combo(values=(industry_list_sorted_good2), default_value='All stocks', 
                  readonly=True, k='-COMBO_ANALYSIS-'),
         ### Need to get list of all industry types
         sg.T('       '), ### spacer
         sg.Input(key='-INPUT_ANALYSIS-', size = (25,1)), sg.Button(button_text='Search', k='-SEARCH_ANALYSIS-'),
            sg.Image(data=sg.DEFAULT_BASE64_LOADING_GIF, enable_events=True, k='-SPIN_ANALYSIS-', visible=False)]
    ]

    frame2_analysis = [
            [sg.T('Stock: ', justification='l',expand_x=True, text_color='white', 
                  k='-TEXT_ANALYSIS_STOCK-'), 
             sg.T('Current price per share: ', justification='l', expand_x=True, text_color='white',
                  k='-TEXT_ANALYSIS_PRICE_PER_SHARE-')],
            [sg.T('Total shares owned: ', justification ='l', expand_x=True, text_color='white', 
                  k='-TEXT_ANALYSIS_SHARES_OWNED-'), 
             sg.T('Highest value past month: ', justification='l', expand_x=True, text_color='white', 
                  k='-TEXT_ANALYSIS_HIGHEST-')],
            [sg.T('Total value of shares: ', justification='l', expand_x=True, text_color='white', 
                  k='-TEXT_ANALYSIS_DOLLARS_OWNED-'),
            sg.T('Lowest value past month: ', justification='l', expand_x=True, text_color='white', 
                 k='-TEXT_ANALYSIS_LOWEST-')]
    ]

    frame3_analysis = [
        [sg.T('Date: ', justification='l',expand_x=True, k='-TEXT_ANALYSIS_DATE-'),
        sg.T('Firm: ', justification='l',expand_x=True, k='-TEXT_ANALYSIS_FIRM-')],
        [sg.T('Recommendation: ', justification = 'c',expand_x=True, pad=((0,0),(10,0)), k='-TEXT_ANALYSIS_RECOMMENDATION-')]
    ]

    frame4_analysis = [
        [sg.T('1. Search stock in top right.\n2. Select buy or sell on bottom left. If selling stock also select stock in bar right beside sell/buy\n3. Select number/price of stock to buy and click convert.\n4. Click confirm',
             justification = 'l')]
    ]
    frame5_analysis = [ 
        [sg.T('Number of shares (decimals ok) or dollar value', justification='c',expand_x=True)],
        [sg.Input('', background_color='red', k='-INPUT_ANALYSIS_STOCKS-'),sg.T('Shares',background_color='blue'),
        sg.Button('CONVERT',k='-BUTTON_ANALYSIS_CONVERT-'), 
        sg.Input('',background_color='red', k='-INPUT_ANALYSIS_DOLLARS-'), sg.T('$',background_color='blue')],
        [sg.Combo(values = ['BUY or SELL?','BUY','SELL'], default_value='BUY or SELL?', readonly=True, k='-COMBO_ANALYSIS_BUY_OR_SELL-'),
         sg.Combo(values = list(portfolio_dict.keys()) + [''], default_value='', visible=False, readonly=True ,k='-COMBO_ANALYSIS_SELL_LIST-'),
        sg.T('Return on investment: % = $', expand_x=True, visible = False, k='-TEXT_ANALYSIS_RETURN-')],
        [sg.Button('CONFIRM', k='-BUTTON_ANALYSIS_CONFIRM-', pad=((700,0),(0,0)))]
    ]


    tab_analysis_layout = [
        [sg.Frame('',frame1_analysis, expand_x=True)],
        [sg.Frame('Stock Info',frame2_analysis, expand_x=True)],
        [sg.Frame('Professional Recommendation',frame3_analysis, expand_x=True)],
        [sg.Frame('Instructions for Buying/Selling Stocks', frame4_analysis, element_justification='l', expand_x=True)],
        [sg.Frame('Buy or Sell Shares',frame5_analysis, expand_x=True, element_justification='c')],
        [sg.Button('Recommendations explained',k='-BUTTON_ANALYSIS_EXPLAINED-')]
    ]

    
    ###################################################################################################
    ###------------------------------------Stock Info Tab-------------------------------------------###
    ###################################################################################################
    ### Stock Info tab
    frame1_info = [
            [sg.Multiline('List of stocks: \n' + mega_stock_name_string, size=(25,3), expand_x=False, expand_y=False, 
                          k='-MLINE_INFO-', pad=(5,5), disabled=True), 
         sg.Combo(values=(industry_list_sorted_good2), default_value='All stocks', 
                  readonly=True, k='-COMBO_INFO-'),
         ### Need to get list of all industry types
         sg.T('       '), ### spacer
         sg.Input(key='-INPUT_INFO-', size=(25,1)), sg.Button(button_text='Search', k='-SEARCH_INFO-')]
    ]

    short_name = ''
    long_name = ''
    country = ''
    state = ''
    city = ''
    industry = ''
    profit_margins = ''
    target_low_price = ''
    rev_per_share = ''
    date_last_updated = ''
    description = ''

    frame2_info = [
            [sg.T('Short Name - ' + short_name, background_color='#4D4D4D', pad=(5,10), 
                  k='SHORT_NAME', size=24, expand_x=True)],
            [sg.T('Country - ' + country, background_color='#4D4D4D', pad=(5,10), 
                  k='COUNTRY', size=24, expand_x=True)], 
            [sg.T('State - ' + state, background_color='#4D4D4D', pad=(5,10),
                  k='STATE', size=24, expand_x=True)], 
            [sg.T('City - ' + city, background_color='#4D4D4D', pad=(5,10),
                  k='CITY', size=24, expand_x=True)],
            [sg.T('Industry - ' + industry, background_color='#4D4D4D', pad=(5,10), 
                  k='INDUSTRY', size=24, expand_x=True)],
            [sg.T('Profit Margins - ' + profit_margins, background_color='#4D4D4D', pad=(5,10), 
                  k='PROFIT_MARGINS', size=24, expand_x=True)], 
            [sg.T('Target Low Price - ' + target_low_price, background_color='#4D4D4D', pad=(5,10), 
                  k='TARGET_LOW_PRICE', size=24, expand_x=True)],
            [sg.T('Revenue Per Share - ' + rev_per_share, background_color='#4D4D4D', pad=(5,10),
                  k='REVENUE_PER_SHARE', size=24, expand_x=True)],
            [sg.T('Last Updated Info on ' + date_last_updated, background_color='#4D4D4D', pad=(5,10),
                  k='DATE_LAST_UPDATED', size=24, expand_x=True)]  

    ]

    col1_info = [
        [sg.Multiline('Company description: \n' + description ,size=(30,3), k='DESCRIPTION', 
                      expand_x=True, expand_y=True,disabled=True)]
    ]

    tab_info_layout = [
        [sg.Frame('',frame1_info, expand_x=True)],
        [sg.Frame('', frame2_info, s=(700,500), expand_x=True, expand_y=True),
        sg.Frame('',col1_info, s=(700,500), expand_x=True,expand_y=True)]
    ]
    
    
    
    ##################################################################################################
    ###------------------------------Stock News Tab------------------------------------------------###
    ##################################################################################################
    ### Stock News tab
    frame1_news = [
            [sg.Multiline('List of stocks: \n' + mega_stock_name_string, size=(25,3), expand_x=False, expand_y=False, 
                          k='-MLINE_NEWS-', pad=(5,5),disabled=True), 
         sg.Combo(values=(industry_list_sorted_good2), default_value='All stocks', 
                  readonly=True, k='-COMBO_NEWS-'),
         ### Need to get list of all industry types
         sg.T('       '), ### spacer
         sg.Input(key='-INPUT_NEWS-', size=(25,1)), sg.Button(button_text='Search', k='-SEARCH_NEWS-')]
    ]

    frame2_news = [
        [sg.Multiline(expand_x=True, expand_y=True, k='-MLINE_NEWS2-', background_color='green', disabled=True, justification='c')],
        [sg.Multiline(expand_x=True, expand_y=True, k='-MLINE_NEWS3-', background_color='red', disabled=True, justification='c')],
        [sg.Multiline(expand_x=True, expand_y=True, k='-MLINE_NEWS4-', background_color='blue', disabled=True, justification='c')]
    ]

    tab_news_layout = [
        [sg.Frame('',frame1_news, expand_x=True)],
        [sg.Frame('',frame2_news, expand_x=True, expand_y=True)]
    ]
    
    
    
    ##################################################################################################
    ###------------------------------------About Tab-----------------------------------------------###
    ##################################################################################################
    with open('about_tab.txt','r', encoding='utf-8') as file:
        about_text = file.read()

    col1_about = [
        [sg.Multiline(about_text, background_color='black', text_color='white', 
                      expand_x=True, expand_y=True, disabled=True),sg.Image(sg.EMOJI_BASE64_HAPPY_HEARTS)]
    ]

    tab_about_layout = [
        [sg.Column(col1_about, expand_x=True, expand_y=True, background_color='black')]
    ]

    
    ##################################################################################################
    ###-------------------------------------Settings Tab-------------------------------------------###
    ##################################################################################################
    col1_settings = [[sg.Button('Remove stock from portfolio', k='-BUTTON_SETTINGS_REMOVE_STOCK-')],
                     [sg.Button('Refresh portfolio page', k='-BUTTON_SETTINGS_REFRESH_PORTFOLIO-')],
                     [sg.Button('Add stock to local info database', k='-BUTTON_SETTINGS_ADD_STOCK_INFO-')],
                    [sg.Button('RESET MONEY MADE', k='-BUTTON_SETTINGS_RESET_MONEY-')]]

    tab_settings_layout = [
        [sg.Column(col1_settings, expand_x=True, expand_y=True, background_color='black')]
    ]

    
    ##################################################################################################
    ###----------------------Defining window and multithreading------------------------------------###
    ##################################################################################################
    sg.theme('black')

    layout = [[sg.TabGroup([[sg.Tab('Portfolio', tab_portfolio_layout),
                            sg.Tab('Stock Performance', tab_performance_layout),
                            sg.Tab('Stock Analysis', tab_analysis_layout),
                            sg.Tab('Stock Info', tab_info_layout),
                             sg.Tab('Stock News', tab_news_layout),
                             sg.Tab('About', tab_about_layout),
                             sg.Tab('Settings', tab_settings_layout)
                            ]], 
                           tooltip='TIP2')]]     

    ### Create the window
    window = sg.Window(
        "Stock Portfolio Interface",
        layout,
        location=(0, 0),
        finalize=True,
        element_justification="center",
        font="Helvetica 18",
        force_toplevel=True,
        resizable=True
    )

    ### Add the plots to the window
    portfolio_fig = my_func.portfolio_pie_plot()
    portfolio_fig_agg = my_func.draw_figure(window['-CANVAS_PORTFOLIO-'].TKCanvas, portfolio_fig)


    ### CHANGE THE DEFAULT OF PLOTTING THE STOCK AMAZON HERE TO THE ENTIRE PORTFOLIO!!!!
    if len(portfolio_dict) > 0:
        portfolio_fig2 = my_func.plot_portfolio_stock(stock='AMZN~1')
    else:
        portfolio_fig2 = my_func.blank_stock_portfolio_performance_plot()
    portfolio_fig_agg2 = my_func.draw_figure(window['-CANVAS_PORTFOLIO2-'].TKCanvas, portfolio_fig2)

    performance_stock,performance_fig = my_func.plot_stock_prices2(period='1d', 
                                                       interval='1m', stock='pick_random_stock')
    performance_fig_agg = my_func.draw_figure(window['-CANVAS_PERFORMANCE-'].TKCanvas, performance_fig)



    ### add the threading information
    mainthread_queue:queue.Queue = None
    #global mainthread_queue
    mainthread_queue = queue.Queue()

    analysis_indicator=0
    
    
    
    ##################################################################################################
    ###----------------------------------Event Loop------------------------------------------------###
    ##################################################################################################
    ### Event loop, where the actions happen
    while True:
        ### First read event is to substantiate a previous state that can be compared to in order to detect changes.
        event, values = window.read(timeout=1)
        old_val_combo_portfolio = values['-COMBO_PORTFOLIO-']
        old_val_combo_performance = values['-COMBO_PERFORMANCE-']
        old_val_combo_analysis = values['-COMBO_ANALYSIS-'] 
        old_val_combo_info = values['-COMBO_INFO-']
        old_val_combo_news = values['-COMBO_NEWS-']

        old_val_input_analysis_stocks = values['-INPUT_ANALYSIS_STOCKS-']
        old_val_input_analysis_dollars = values['-INPUT_ANALYSIS_DOLLARS-']

        event, values = window.read(timeout=100)
        ### End program if user closes window

        ### Making the filter for all the stock tabs in the dropdown menu
        if values['-COMBO_PERFORMANCE-'] != old_val_combo_performance:
            try:
                update_str = str('List of stocks: ' + values['-COMBO_PERFORMANCE-'] + '\n' + industry_stocks_dict[values['-COMBO_PERFORMANCE-']])
                window['-MLINE_PERFORMANCE-'].Update(update_str)
            except TypeError: 
                break

        if values['-COMBO_ANALYSIS-'] != old_val_combo_analysis:
            update_str = str('List of stocks: ' + values['-COMBO_ANALYSIS-'] + '\n' + industry_stocks_dict[values['-COMBO_ANALYSIS-']])
            window['-MLINE_ANALYSIS-'].Update(update_str)

        if values['-COMBO_INFO-'] != old_val_combo_info:
            update_str = str('List of stocks: ' + values['-COMBO_INFO-'] + '\n' + industry_stocks_dict[values['-COMBO_INFO-']])
            window['-MLINE_INFO-'].Update(update_str)

        if values['-COMBO_NEWS-'] != old_val_combo_news:
            update_str = str('List of stocks: ' + values['-COMBO_NEWS-'] + '\n' + industry_stocks_dict[values['-COMBO_NEWS-']])
            window['-MLINE_NEWS-'].Update(update_str)

            
        ###############################################################################################
        ### ------------------------------PORTFOLIO TAB EVENT LOOP----------------------------------###
        ###############################################################################################
        if values['-COMBO_PORTFOLIO-'] != old_val_combo_portfolio:
            print('changing the portfolio graph')
            stock = values['-COMBO_PORTFOLIO-']
            portfolio_fig2 = my_func.plot_portfolio_stock(stock=stock)
            my_func.close_figure(portfolio_fig_agg2)
            portfolio_fig_agg2 = my_func.draw_figure(window['-CANVAS_PORTFOLIO2-'].TKCanvas, portfolio_fig2)

            
        ###############################################################################################
        ### ---------------------------STOCK PERFORMANCE TAB EVENT LOOP-----------------------------###
        ###############################################################################################
        ### Search button to search for a new stock
        if event == '-SEARCH_PERFORMANCE-':
            performance_search_stock = values['-INPUT_PERFORMANCE-'].upper()
            performance_stock, performance_fig = my_func.plot_stock_prices2(period='1d',interval='1m',
                                                                           stock=performance_search_stock)
            my_func.close_figure(performance_fig_agg)
            performance_fig_agg = my_func.draw_figure(window["-CANVAS_PERFORMANCE-"].TKCanvas, 
                                                      performance_fig)    

        ### Button mapping to change the stock performance graph timescales
        if event == '-BUTTON_PERFORMANCE_TODAY-':
            performance_stock, performance_fig = my_func.plot_stock_prices2(period='1d',interval='1m',
                                                                           stock=performance_stock)
            my_func.close_figure(performance_fig_agg)
            performance_fig_agg = my_func.draw_figure(window["-CANVAS_PERFORMANCE-"].TKCanvas, 
                                                      performance_fig)

        if event == '-BUTTON_PERFORMANCE_1WEEK-':
            performance_stock, performance_fig = my_func.plot_stock_prices2(period='5d',interval='1m',
                                                                           stock=performance_stock)
            my_func.close_figure(performance_fig_agg)
            performance_fig_agg = my_func.draw_figure(window["-CANVAS_PERFORMANCE-"].TKCanvas, 
                                                      performance_fig)

        if event == '-BUTTON_PERFORMANCE_1MONTH-':
            performance_stock, performance_fig = my_func.plot_stock_prices2(period='1mo',interval='5m',
                                                                           stock=performance_stock)
            my_func.close_figure(performance_fig_agg)
            performance_fig_agg = my_func.draw_figure(window["-CANVAS_PERFORMANCE-"].TKCanvas, 
                                                      performance_fig)

        if event == '-BUTTON_PERFORMANCE_6MONTHS-':
            performance_stock, performance_fig = my_func.plot_stock_prices2(period='6mo',interval='1h',
                                                                           stock=performance_stock)
            my_func.close_figure(performance_fig_agg)
            performance_fig_agg = my_func.draw_figure(window["-CANVAS_PERFORMANCE-"].TKCanvas, 
                                                      performance_fig)

        if event == '-BUTTON_PERFORMANCE_1YEAR-':
            performance_stock, performance_fig = my_func.plot_stock_prices2(period='1y',interval='1h',
                                                                           stock=performance_stock)
            my_func.close_figure(performance_fig_agg)
            performance_fig_agg = my_func.draw_figure(window["-CANVAS_PERFORMANCE-"].TKCanvas, 
                                                      performance_fig) 

        ###############################################################################################
        ### -----------------------------STOCK ANALYSIS TAB EVENT LOOP------------------------------###
        ###############################################################################################
        analysis_thread_indicator = 0
        ### If a stock is searched
        if event == '-SEARCH_ANALYSIS-':
            stock_key_analysis = values['-INPUT_ANALYSIS-'].upper()

            ### Resets the rest of the window so don't run into other issues
            window['-INPUT_ANALYSIS_DOLLARS-'].update('')
            window['-INPUT_ANALYSIS_STOCKS-'].update('')
            window['-COMBO_ANALYSIS_BUY_OR_SELL-'].update('BUY or SELL?')
            window['-COMBO_ANALYSIS_SELL_LIST-'].update(visible=False)
            window['-TEXT_ANALYSIS_RETURN-'].update(visible=False)

            if stock_key_analysis in stock_info_dict.keys():
                try: ### For if the stock symbol provided is not actually a stock
                    stock_key_analysis = values['-INPUT_ANALYSIS-'].upper()
                    analysis_thread_indicator=1

                    print('starting thread')
                    stock_data, stock_lowest, stock_highest, recommendation_df = my_func.stock_analysis_tab_data_retriever(stock_key_analysis) 

                    if stock_data != 'Analysis retrieval failed':
                        thread_id = threading.Thread(target=my_func.stock_analysis_tab_data_retriever_thread, 
                                                 name='stock_analysis_thread', daemon=True, 
                                                 args=(analysis_indicator, mainthread_queue, stock_key_analysis, 
                                                       stock_data,stock_lowest,stock_highest, recommendation_df))
                        thread_id.start()
                    
                    ### Notify the user the first time they open the program and the analysis function does not work
                    if ((stock_data == 'Analysis retrieval failed') and (notified_analysis_broken == False)):
                        file = open('analysis_broken.txt','r', encoding='utf-8')
                        text = file.read()
                        file.close()
                        sg.popup_scrolled(text, title="Analysis data retriever broken :(")
                        notified_analysis_broken = True

                    if stock_data == 'Analysis retrieval failed':
                        analysis_thread_indicator=0 ### This resets the counter that sees a stock being analyzed
                        stock_data = yfi.ticker.Ticker(stock_key_analysis).history(period='1mo',interval='5m')
                        stock_lowest = np.round(np.sort(stock_data['Low'])[0],2)
                        stock_highest = np.round(np.sort(stock_data['High'])[-1],2)

                    ### Printing out the values that are given if the analysis works
                    shortname = stock_info_dict[stock_key_analysis]['shortName']
                    window['-TEXT_ANALYSIS_STOCK-'].update(f'Stock: {stock_key_analysis} -- {shortname}')
                    current_share_price = np.round(stock_data.iloc[-1]['Close'], 2)
                    window['-TEXT_ANALYSIS_PRICE_PER_SHARE-'].update(f'Current price per share: ${current_share_price}')

                    num_shares_text = ''
                    num_shares = 0
                    i=0
                    for key in portfolio_dict:
                        if re.match(stock_key_analysis + '~\d', key) != None:
                            num_shares_text = num_shares_text + key[-1] + '~' + str(portfolio_dict[key]['num_stocks']) + '   '    
                            num_shares = num_shares + portfolio_dict[key]['num_stocks']
                            i+=1
                    if i==0:
                        num_shares_text='0'

                    window['-TEXT_ANALYSIS_SHARES_OWNED-'].update(f'Total shares owned: {num_shares_text}')
                    window['-TEXT_ANALYSIS_HIGHEST-'].update(f'Highest value past month: ${stock_highest}')

                    dollars_owned = np.round(current_share_price*num_shares,2)
                    window['-TEXT_ANALYSIS_DOLLARS_OWNED-'].update(f'Total value of shares: ${dollars_owned}')
                    window['-TEXT_ANALYSIS_LOWEST-'].update(f'Lowest value past month: ${stock_lowest}')
                
                    sell_combo_list = []
                    for key in portfolio_dict:
                        if re.match(stock_key_analysis + '~\d', key) != None:
                            sell_combo_list.append(key)
                    window['-COMBO_ANALYSIS_SELL_LIST-'].update(sell_combo_list)
                except KeyError:
                    sg.Popup('''Stock symbol entered is either incorrect or not in the local database.
                             Contact owner if stock is missing or make a github issue. \n
                             l.v.denoncourt@gmail.com''')
            else:
                sg.Popup('''Stock symbol entered is either incorrect or not in the local database.
                             Contact owner if stock is missing or make a github issue. \n
                             l.v.denoncourt@gmail.com''') 

        ### If the explain recommendations button is pressed
        if event == '-BUTTON_ANALYSIS_EXPLAINED-':
            #print('entered if')
            file = open('recommendations_explained.txt','r', encoding='utf-8')
            data = file.read()
            file.close()
            #data
            sg.popup_scrolled(data)

    ### Receiver of the thread messages - NEED TO MAKE SURE CAN READ MESSAGE AND GET MORE THAN ONE THREAD AND DELINEATE
        try:
            message = mainthread_queue.get_nowait()
            print(message)
        except queue.Empty:
            message = None
        if message is not None:
            recommendation_df = message[3]
            window['-TEXT_ANALYSIS_DATE-'].update(f'Date: {recommendation_df.iloc[len(recommendation_df)-1][0]}')
            window['-TEXT_ANALYSIS_FIRM-'].update(f'Firm: {recommendation_df.iloc[len(recommendation_df)-1][1]}')
            window['-TEXT_ANALYSIS_RECOMMENDATION-'].update(f'Recommendation: {recommendation_df.iloc[len(recommendation_df)-1][2]}')

            window['-SPIN_ANALYSIS-'].update(visible=False)
            analysis_thread_indicator = 0 

        ### converting num_shares to dollars and vise versa
        if (values['-INPUT_ANALYSIS_STOCKS-'] != old_val_input_analysis_stocks) and (values['-INPUT_ANALYSIS-'] != '') and (values['-COMBO_ANALYSIS_SELL-']):
            if values['-INPUT_ANALYSIS_STOCKS-'] != '':
                price = np.round(float(values['-INPUT_ANALYSIS_STOCKS-'])*float(current_share_price),2)
            else:
                price = ''
            window['-INPUT_ANALYSIS_DOLLARS-'].update(price)
        if (values['-INPUT_ANALYSIS_DOLLARS-'] != old_val_input_analysis_dollars) and (values['-INPUT_ANALYSIS-'] != ''):
            if values['-INPUT_ANALYSIS_DOLLARS-'] != '':
                num_stocks = np.round(float(values['-INPUT_ANALYSIS_DOLLARS-'])/float(current_share_price),2)
            else:
                window['-INPUT_ANALYSIS_STOCKS-'].update(num_stocks)
        if (((values['-INPUT_ANALYSIS_STOCKS-'] != old_val_input_analysis_stocks) or (values['-INPUT_ANALYSIS_DOLLARS-'] != old_val_input_analysis_dollars)) and (values['-INPUT_ANALYSIS-'] == '')):
            sg.Popup('Please search a stock in the top right before attempting to buy or sell', title='Uh-oh')  
            window['-INPUT_ANALYSIS_DOLLARS-'].update('')
            window['-INPUT_ANALYSIS_STOCKS-'].update('')
                
                
        ### If select SELL, then show the rest of the sell options and text
        if values['-COMBO_ANALYSIS_BUY_OR_SELL-'] == 'SELL':
            window['-COMBO_ANALYSIS_SELL_LIST-'].update(visible=True)
        else:
            window['-COMBO_ANALYSIS_SELL_LIST-'].update(visible=False)
            window['-TEXT_ANALYSIS_RETURN-'].update(visible=False)

        if (values['-COMBO_ANALYSIS_BUY_OR_SELL-'] == 'SELL') & (values['-COMBO_ANALYSIS_SELL_LIST-'] != '') & (values['-INPUT_ANALYSIS_STOCKS-'] != '') & (values['-INPUT_ANALYSIS_DOLLARS-'] != ''):  
            sell_amount_stock = float(values['-INPUT_ANALYSIS_STOCKS-'])
            sell_amount_dollars = float(values['-INPUT_ANALYSIS_DOLLARS-'])
            stock_key_sell_list_analysis = values['-COMBO_ANALYSIS_SELL_LIST-']

            num_stocks_total = float(portfolio_dict[stock_key_sell_list_analysis]['num_stocks'])
            price_bought_at =float(portfolio_dict[stock_key_sell_list_analysis]['price_bought_at'])

            if num_stocks_total < sell_amount_stock:
                sg.Popup('The number of shares you are trying to sell is greater than the number of shares that you own!', title='Uh-oh')
                window['-INPUT_ANALYSIS_DOLLARS-'].update('')
                window['-INPUT_ANALYSIS_STOCKS-'].update('')
            else:
                original_share_sell_value = price_bought_at*sell_amount_stock
                percent_return = np.round((sell_amount_dollars-original_share_sell_value)/original_share_sell_value*100,2)
                if percent_return > 0:
                    percent_return = '+' + str(percent_return)

                dollars_return = np.round(sell_amount_dollars - original_share_sell_value,2)

                update_str = (f'Return on investment: {percent_return}% = ${dollars_return}')
                window['-TEXT_ANALYSIS_RETURN-'].update(update_str)
                window['-TEXT_ANALYSIS_RETURN-'].update(visible=True)

        ### Confirming buying or selling
        if event == '-BUTTON_ANALYSIS_CONFIRM-':
            ### BUYING A STOCK
            if (values['-COMBO_ANALYSIS_BUY_OR_SELL-'] == 'BUY') & (values['-INPUT_ANALYSIS_STOCKS-'] != '') & (values['-INPUT_ANALYSIS_DOLLARS-'] != ''):
                month = str(datetime.date.today().month)
                if len(month) == 1:
                    month  = '0' + month
                day = str(datetime.date.today().day)
                if len(day) == 1:
                    day  = '0' + day
                year = str(datetime.date.today().year)
                date = year + '-' + month + '-' + day

                for a in np.arange(1,100):
                    if stock_key_analysis + '~' + str(a) not in list(portfolio_dict.keys()):
                        stock_sym_dict = stock_key_analysis + '~' + str(a)
                        break
                    else:
                        pass

                dict_entry = {'price_bought_at': np.round(current_share_price,2), 'date':date, 'num_stocks':float(values['-INPUT_ANALYSIS_STOCKS-'])}
                portfolio_dict[stock_sym_dict] = dict_entry
                ### Sort the dictionary, save it, and reload it
                portfolio_dict2 = {}
                for a in list(np.sort(list(portfolio_dict.keys()))):
                    portfolio_dict2[a] = portfolio_dict[a]
                portfolio_dict = portfolio_dict2

                with open('stock_portfolio_dict_TEST.p', 'wb') as handle:
                    pickle.dump(portfolio_dict, handle)

                with open('stock_portfolio_dict_TEST.p', 'rb') as handle:
                    portfolio_dict = pickle.load(handle)
                
                num_shares_text = ''
                num_shares = 0
                i=0
                for key in portfolio_dict:
                    if re.match(stock_key_analysis + '~\d', key) != None:
                        num_shares_text = num_shares_text + key[-1] + '~' + str(portfolio_dict[key]['num_stocks']) + '   '    
                        num_shares = num_shares + portfolio_dict[key]['num_stocks']
                        i+=1
                if i==0:
                    num_shares_text='0'

                window['-TEXT_ANALYSIS_SHARES_OWNED-'].update(f'Total shares owned: {num_shares_text}')
                dollars_owned = np.round(current_share_price*num_shares,2)
                window['-TEXT_ANALYSIS_DOLLARS_OWNED-'].update(f'Total value of shares: ${dollars_owned}')
                
                sell_combo_list = []
                for key in portfolio_dict:
                    if re.match(stock_key_analysis + '~\d', key) != None:
                        sell_combo_list.append(key)
                window['-COMBO_ANALYSIS_SELL_LIST-'].update(sell_combo_list)
                
                ### Updating the entire portfolio tab
                sg.execute_py_file('refreshing_portfolio_loading_gifs.py')
                portfolio_fig = my_func.portfolio_pie_plot()
                my_func.close_figure(portfolio_fig_agg)
                portfolio_fig_agg = my_func.draw_figure(window['-CANVAS_PORTFOLIO-'].TKCanvas, portfolio_fig)

                if len(portfolio_dict) > 0:
                    portfolio_fig2 = my_func.plot_portfolio_stock(stock='AMZN~1')
                else:
                    portfolio_fig2 = my_func.blank_stock_portfolio_performance_plot()
                my_func.close_figure(portfolio_fig_agg2)
                portfolio_fig_agg2 = my_func.draw_figure(window['-CANVAS_PORTFOLIO2-'].TKCanvas, portfolio_fig2)

                window['-COMBO_PORTFOLIO-'].update(values = list(portfolio_dict.keys()) + ['Portfolio'])

                text_list = my_func.update_portfolio_text()
                window['-MULTILINE_PORTFOLIO-'].update(text_list)
                with open('finished_flag.p', 'wb') as f:
                    pickle.dump('Finished',f)



            ### SELLING SHARES    
            if (values['-COMBO_ANALYSIS_BUY_OR_SELL-'] == 'SELL') & (values['-INPUT_ANALYSIS_STOCKS-'] != '') & (values['-INPUT_ANALYSIS_DOLLARS-'] != '') & (values['-COMBO_ANALYSIS_SELL_LIST-'] != ''): 
                stock_key_sell_list_analysis = values['-COMBO_ANALYSIS_SELL_LIST-']
                num_stock_sell = float(values['-INPUT_ANALYSIS_STOCKS-'])
                stocks_owned = float(portfolio_dict[stock_key_sell_list_analysis]['num_stocks'])
                ### IF SELLING ALL SHARES
                if num_stock_sell == stocks_owned:
                    portfolio_dict.pop(stock_key_sell_list_analysis)

                    sell_current_value = values['-INPUT_ANALYSIS_DOLLARS-']
                    money_made = float(money_made) + float(sell_current_value)
                    money_made = str(money_made)

                    with open('money_made.txt','w', encoding='utf-8') as file:
                        file.write(money_made)

                    ### Updating the saved portfolio
                    with open('stock_portfolio_dict_TEST.p', 'wb') as handle:
                        pickle.dump(portfolio_dict, handle)
                    with open('stock_portfolio_dict_TEST.p', 'rb') as handle:
                        portfolio_dict = pickle.load(handle)

                    ### Resets the rest of the window so don't run into other issues
                    window['-INPUT_ANALYSIS_DOLLARS-'].update('')
                    window['-INPUT_ANALYSIS_STOCKS-'].update('')
                    window['-COMBO_ANALYSIS_BUY_OR_SELL-'].update('BUY or SELL?')
                    window['-COMBO_ANALYSIS_SELL_LIST-'].update(visible=False)
                    window['-TEXT_ANALYSIS_RETURN-'].update(visible=False)
                    
                    ### Updating the shares owned and the value of those shares
                    num_shares_text = ''
                    num_shares = 0
                    i=0
                    for key in portfolio_dict:
                        if re.match(stock_key_analysis + '~\d', key) != None:
                            num_shares_text = num_shares_text + key[-1] + '~' + str(portfolio_dict[key]['num_stocks']) + '   '    
                            num_shares = num_shares + portfolio_dict[key]['num_stocks']
                            i+=1
                    if i==0:
                        num_shares_text='0'
                    
                    window['-TEXT_ANALYSIS_SHARES_OWNED-'].update(f'Total shares owned: {num_shares_text}')
                    dollars_owned = np.round(current_share_price*num_shares,2)
                    window['-TEXT_ANALYSIS_DOLLARS_OWNED-'].update(f'Total value of shares: ${dollars_owned}')
                    
                    ### Updating the sell list combo
                    sell_combo_list = []
                    for key in portfolio_dict:
                        if re.match(stock_key_analysis + '~\d', key) != None:
                            sell_combo_list.append(key)
                    window['-COMBO_ANALYSIS_SELL_LIST-'].update(sell_combo_list)
                    
                    ### Updating the entire portfolio tab
                    sg.execute_py_file('refreshing_portfolio_loading_gifs.py')
                    portfolio_fig = my_func.portfolio_pie_plot()
                    my_func.close_figure(portfolio_fig_agg)
                    portfolio_fig_agg = my_func.draw_figure(window['-CANVAS_PORTFOLIO-'].TKCanvas, portfolio_fig)

                    if len(portfolio_dict) > 0:
                        portfolio_fig2 = my_func.plot_portfolio_stock(stock='AMZN~1')
                    else:
                        portfolio_fig2 = my_func.blank_stock_portfolio_performance_plot()
                    my_func.close_figure(portfolio_fig_agg2)
                    portfolio_fig_agg2 = my_func.draw_figure(window['-CANVAS_PORTFOLIO2-'].TKCanvas, portfolio_fig2)

                    window['-COMBO_PORTFOLIO-'].update(values = list(portfolio_dict.keys()) + ['Portfolio'])

                    text_list = my_func.update_portfolio_text()
                    window['-MULTILINE_PORTFOLIO-'].update(text_list)
                    with open('finished_flag.p', 'wb') as f:
                        pickle.dump('Finished',f)
                ### IF SELLING NOT ALL OF THE SHARES
                else:
                    sell_current_value = values['-INPUT_ANALYSIS_DOLLARS-']
                    money_made = float(money_made) + float(sell_current_value)
                    money_made = str(money_made)
                    portfolio_dict[stock_key_analysis]['num_stocks'] = float(portfolio_dict[stock_key_analysis]['num_stocks']) - num_stock_sell

                    ### Updating the saved portfolio and money made
                    with open('stock_portfolio_dict_TEST.p', 'wb') as handle:
                        pickle.dump(portfolio_dict, handle)
                    with open('stock_portfolio_dict_TEST.p', 'rb') as handle:
                        portfolio_dict = pickle.load(handle)
                    with open('money_made.txt','w', encoding='utf-8') as file:
                        file.write(money_made)


                    ### Resets the rest of the window so don't run into other issues
                    window['-INPUT_ANALYSIS_DOLLARS-'].update('')
                    window['-INPUT_ANALYSIS_STOCKS-'].update('')
                    window['-COMBO_ANALYSIS_BUY_OR_SELL-'].update('BUY or SELL?')
                    window['-COMBO_ANALYSIS_SELL_LIST-'].update(visible=False)
                    window['-TEXT_ANALYSIS_RETURN-'].update(visible=False)

                    ### Updating the shares owned and the value of all those shares
                    num_shares_text = ''
                    num_shares = 0
                    i=0
                    for key in portfolio_dict:
                        print(key)
                        print(stock_key_analysis)
                        if re.match(stock_key_analysis + '~\d', key) != None:
                            print('entering the portfolio_dict loop')
                            num_shares_text = num_shares_text + key[-1] + '~' + str(portfolio_dict[key]['num_stocks']) + '   '    
                            num_shares = num_shares + portfolio_dict[key]['num_stocks']
                            i+=1
                    if i==0:
                        num_shares_text='0'
                    print('')
                    print(num_shares_text)
                    print('')
                    window['-TEXT_ANALYSIS_SHARES_OWNED-'].update(f'Total shares owned: {num_shares_text}')
                    dollars_owned = np.round(current_share_price*num_shares,2)
                    window['-TEXT_ANALYSIS_DOLLARS_OWNED-'].update(f'Total value of shares: ${dollars_owned}')

                    ### Updating the sell combo list 
                    sell_combo_list = []
                    for key in portfolio_dict:
                        if re.match(stock_key_analysis + '~\d', key) != None:
                            sell_combo_list.append(key)
                    window['-COMBO_ANALYSIS_SELL_LIST-'].update(sell_combo_list)

                    ### Updating the entire portfolio tab
                    sg.execute_py_file('refreshing_portfolio_loading_gifs.py')
                    portfolio_fig = my_func.portfolio_pie_plot()
                    my_func.close_figure(portfolio_fig_agg)
                    portfolio_fig_agg = my_func.draw_figure(window['-CANVAS_PORTFOLIO-'].TKCanvas, portfolio_fig)

                    if len(portfolio_dict) > 0:
                        portfolio_fig2 = my_func.plot_portfolio_stock(stock='AMZN~1')
                    else:
                        portfolio_fig2 = my_func.blank_stock_portfolio_performance_plot()
                    my_func.close_figure(portfolio_fig_agg2)
                    portfolio_fig_agg2 = my_func.draw_figure(window['-CANVAS_PORTFOLIO2-'].TKCanvas, portfolio_fig2)

                    window['-COMBO_PORTFOLIO-'].update(values = list(portfolio_dict.keys()) + ['Portfolio'])

                    text_list = my_func.update_portfolio_text()
                    window['-MULTILINE_PORTFOLIO-'].update(text_list)
                    with open('finished_flag.p', 'wb') as f:
                        pickle.dump('Finished',f)


            if (((values['-INPUT_ANALYSIS_STOCKS-'] == '')  ^ (values['-INPUT_ANALYSIS_DOLLARS-'] == '')) or (values['-INPUT_ANALYSIS-'] == '') or (values['-COMBO_ANALYSIS_BUY_OR_SELL-'] == 'BUY or SELL?') and (values['-COMBO_ANALYSIS_SELL_LIST-']=='')):   
                sg.Popup('Erroneous input. Please refer to the instructions when attempting to buy or sell a stock', title='Uh=oh')


        ###############################################################################################
        ### ----------------------------STOCK INFO TAB EVENT LOOP-----------------------------------###
        ###############################################################################################
        ### Making the search bar work 
        if event == '-SEARCH_INFO-':
            try:
                stock_key_info = values['-INPUT_INFO-'].upper()
                short_name = str(stock_info_dict[stock_key_info]['shortName'])
                description = str(stock_info_dict[stock_key_info]['longBusinessSummary'])
                state = str(stock_info_dict[stock_key_info]['state'])
                city = str(stock_info_dict[stock_key_info]['city'])
                country = str(stock_info_dict[stock_key_info]['country'])
                profit_margins = str(stock_info_dict[stock_key_info]['profitMargins'])
                target_low_price = str(stock_info_dict[stock_key_info]['targetLowPrice'])
                rev_per_share = str(stock_info_dict[stock_key_info]['revenuePerShare'])
                industry = str(stock_info_dict[stock_key_info]['industry'])
                date_last_updated = str(stock_info_dict[stock_key_info]['date_last_updated'])

                #print('made it past the variable assignments')
                window['SHORT_NAME'].update('Short Name - ' + short_name)
                window['DESCRIPTION'].update('Company description: \n' + description)
                window['COUNTRY'].update('Country - ' + country)
                window['STATE'].update('State - ' + states_abrv[states_abrv['code'] == state]['state'].values[0])
                window['CITY'].update('City - ' + city)
                window['PROFIT_MARGINS'].update('Profit Margins - $' + profit_margins)
                window['TARGET_LOW_PRICE'].update('Target Low Price - $' + target_low_price)
                window['REVENUE_PER_SHARE'].update('Revenue Per Share - $' + rev_per_share)
                window['INDUSTRY'].update('Industry - ' + industry)
                window['DATE_LAST_UPDATED'].update('Last Updated Info on ' + date_last_updated)
                #print('made it past the window changes')
            except KeyError:
                sg.Popup('''Stock symbol entered is either incorrect or not in the local database.
                         Contact owner if stock is missing or make a github issue. \n
                         l.v.denoncourt@gmail.com''')    


        ###############################################################################################
        ### ----------------------------STOCK NEWS TAB EVENT LOOP-----------------------------------###
        ###############################################################################################
        if event == '-SEARCH_NEWS-':
            #print('entered event loop')
            stock_key_news = values['-INPUT_NEWS-'].upper()
            #print(stock_key)
            news_list = yfi.ticker.Ticker(stock_key_news).news
            try:
                news_art1 = news_list[0]
                #print(news_art1)
                date_news_unformated = datetime.datetime.fromtimestamp(news_art1['providerPublishTime'])
                date_news = date_news_unformated.strftime('%h/%d/%Y %r')
                window['-MLINE_NEWS2-'].update('TITLE: ' + news_art1['title'] + '\nPUBLISHER: ' + news_art1['publisher'] + '\nDATE PUBLISHED: ' + date_news +'\nURL: ' + news_art1['link'])

                news_art2 = news_list[1]
                #print(news_art1)
                date_news_unformated = datetime.datetime.fromtimestamp(news_art2['providerPublishTime'])
                date_news = date_news_unformated.strftime('%h/%d/%Y %r')
                window['-MLINE_NEWS3-'].update('TITLE: ' + news_art2['title'] + '\nPUBLISHER: ' + news_art2['publisher'] + '\nDATE PUBLISHED: ' + date_news +'\nURL: ' + news_art2['link'])

                news_art3 = news_list[2]
                #print(news_art1)
                date_news_unformated = datetime.datetime.fromtimestamp(news_art3['providerPublishTime'])
                date_news = date_news_unformated.strftime('%h/%d/%Y %r')
                window['-MLINE_NEWS4-'].update('TITLE: ' + news_art3['title'] + '\nPUBLISHER: ' + news_art3['publisher'] + '\nDATE PUBLISHED: ' + date_news +'\nURL: ' + news_art3['link'])  
            except IndexError: # Means that there isnt any news articles (I think, haven't tested)
                pass
            except KeyError:
                sg.Popup('''Stock symbol entered is either incorrect or not in the local database.
                         Contact owner if stock is missing or make a github issue. \n
                         l.v.denoncourt@gmail.com''')


        ###############################################################################################
        ### --------------------------------Setting TAB EVENT LOOP---------------------------------###
        ###############################################################################################
        if event == '-BUTTON_SETTINGS_REMOVE_STOCK-':
            remove_stock_window = sg.Window('',[[sg.T('Select what stock you want to remove', k='_')],
                                      [sg.Combo(values = list(portfolio_dict.keys()) + ['All stocks',''],
                                                k='-COMBO_SETTINGS_REMOVE_STOCK-',default_value='')],
                                      [sg.Button('Delete', k='-BUTTON_SETTINGS_DELETE_STOCK-')]
                                      ])
            while True:            
                event_remove_stock, values_remove_stock = remove_stock_window.read(timeout=100)
                if event_remove_stock == '-BUTTON_SETTINGS_DELETE_STOCK-':
                    stock_to_remove = values_remove_stock['-COMBO_SETTINGS_REMOVE_STOCK-']
                    if stock_to_remove == 'All stocks':
                        portfolio_dict = {}
                    else:
                        portfolio_dict.pop(stock_to_remove) ### Remove the stock from the dictionary

                    ### Save the dictionary change and reload it
                    with open('stock_portfolio_dict_TEST.p', 'wb') as handle:
                        pickle.dump(portfolio_dict, handle)
                    with open('stock_portfolio_dict_TEST.p', 'rb') as handle:
                        portfolio_dict = pickle.load(handle)
                    remove_stock_window['-COMBO_SETTINGS_REMOVE_STOCK-'].update(values = list(portfolio_dict.keys()) + ['All stocks','']) 

                    ### Updating the entire portfolio tab
                    sg.execute_py_file('refreshing_portfolio_loading_gifs.py')
                    portfolio_fig = my_func.portfolio_pie_plot()
                    my_func.close_figure(portfolio_fig_agg)
                    portfolio_fig_agg = my_func.draw_figure(window['-CANVAS_PORTFOLIO-'].TKCanvas, portfolio_fig)

                    if len(portfolio_dict) > 0:
                        portfolio_fig2 = my_func.plot_portfolio_stock(stock='AMZN~1')
                    else:
                        portfolio_fig2 = my_func.blank_stock_portfolio_performance_plot()
                    my_func.close_figure(portfolio_fig_agg2)
                    portfolio_fig_agg2 = my_func.draw_figure(window['-CANVAS_PORTFOLIO2-'].TKCanvas, portfolio_fig2)

                    window['-COMBO_PORTFOLIO-'].update(values = list(portfolio_dict.keys()) + ['Portfolio'])

                    text_list = my_func.update_portfolio_text()
                    window['-MULTILINE_PORTFOLIO-'].update(text_list)
                    with open('finished_flag.p', 'wb') as f:
                        pickle.dump('Finished',f)

                    event_remove_stock, values_remove_stock = remove_stock_window.read(timeout=1)
                if event_remove_stock == sg.WIN_CLOSED:
                    break 
            if event == sg.WIN_CLOSED:
                break 

        if event == '-BUTTON_SETTINGS_REFRESH_PORTFOLIO-':
            ### Updating the entire portfolio tab
            sg.execute_py_file('refreshing_portfolio_loading_gifs.py')
            portfolio_fig = my_func.portfolio_pie_plot()
            my_func.close_figure(portfolio_fig_agg)
            portfolio_fig_agg = my_func.draw_figure(window['-CANVAS_PORTFOLIO-'].TKCanvas, portfolio_fig)

            if len(portfolio_dict) > 0:
                portfolio_fig2 = my_func.plot_portfolio_stock(stock='AMZN~1')
            else:
                portfolio_fig2 = my_func.blank_stock_portfolio_performance_plot()
            my_func.close_figure(portfolio_fig_agg2)
            portfolio_fig_agg2 = my_func.draw_figure(window['-CANVAS_PORTFOLIO2-'].TKCanvas, portfolio_fig2)

            window['-COMBO_PORTFOLIO-'].update(values = list(portfolio_dict.keys()) + ['Portfolio'])

            text_list = my_func.update_portfolio_text()
            window['-MULTILINE_PORTFOLIO-'].update(text_list)
            with open('finished_flag.p', 'wb') as f:
                pickle.dump('Finished',f)
    window.close()

    
    
if __name__ == '__main__':
    main()

AMZN: 1m data not available for startTime=1674277200 and endTime=1681932029. Only 7 days worth of 1m granularity data are allowed to be fetched per request.
AMZN: 2m data not available for startTime=1674277200 and endTime=1681932029. The requested range must be within the last 60 days.
AMZN: 5m data not available for startTime=1674277200 and endTime=1681932029. The requested range must be within the last 60 days.
AMZN: 15m data not available for startTime=1674277200 and endTime=1681932030. The requested range must be within the last 60 days.
AMZN: 15m data not available for startTime=1674277200 and endTime=1681932030. The requested range must be within the last 60 days.


C:\Users\lvden\Documents\Luke stuff\stock_portfolio\Stock_portfolio_functions2.py:386: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dow_df['Datetime'].iloc[i] = a.replace(tzinfo=None)
C:\Users\lvden\Documents\Luke stuff\stock_portfolio\Stock_portfolio_functions2.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dow_df['Datetime'].iloc[i] = a.replace(tzinfo=None)


starting thread
Analysis retrieval failed
